In [1]:
import pandas as pd 
import numpy as np
import re
import sys, getopt
import csv
import os
import matplotlib.pyplot as plt
import seaborn as sns

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

os.chdir('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings')

os.getcwd()

ModuleNotFoundError: No module named 'pandas'

In [ ]:
#this is making use of code first written in plotting_overall_intervals.ipynb

# MARKET VALUE

## more valuable team winning

In [ ]:
latest_match_data = load_csv_dataset('create_dataset_1/CSVs we use often/latest_match_data_june_18_SUPERSTAR_INFO_ADDED.csv')
latest_match_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)
latest_match_data['mv_difference_abs'] = latest_match_data['Market.Value.Difference'].abs()
# Assuming 'latest_match_data' is already loaded
latest_match_data['MV_Prediction_Correct'] = np.where(
    (latest_match_data['Market.Value.Difference'] < 0) & (latest_match_data['Score.Difference'] < 0), 1,
    np.where(
        (latest_match_data['Market.Value.Difference'] > 0) & (latest_match_data['Score.Difference'] > 0), 1, 0
    )
)
pd.set_option('display.max_columns', None)

# ... existing code ...

# Create a new column for the year pairs
latest_match_data['year_pair'] = latest_match_data['Season'].apply(lambda x: f"{x//2*2}-{x//2*2+1}")

# Define the intervals
intervals = [0, 500000, 1000000, 5000000, 10000000, 50000000, 100000000, 500000000, 1000000000]
labels = ['0-500K', '500K-1M', '1M-5M', '5M-10M', '10M-50M', '50M-100M', '100M-500M', '500M-1B']

# Create a new column for the intervals
latest_match_data['mv_interval'] = pd.cut(latest_match_data['mv_difference_abs'], bins=intervals, labels=labels)

# Initialize an empty DataFrame to store the results
all_interval_stats_more_valuable = pd.DataFrame()

# Group by the year pairs and intervals and calculate the required statistics
for year_pair, group in latest_match_data.groupby('year_pair'):
    interval_stats = group.groupby('mv_interval').agg(
        total_games=('MV_Prediction_Correct', 'size'),
        more_valuable_wins=('MV_Prediction_Correct', 'sum')
    ).reset_index()
    
    # Calculate the percentage of games won by the more valuable team
    interval_stats['win_percentage'] = (interval_stats['more_valuable_wins'] / interval_stats['total_games']) * 100
    
    # Add the year pair to the results
    interval_stats['year_pair'] = year_pair
    
    # Append to the overall results DataFrame
    all_interval_stats_more_valuable = pd.concat([all_interval_stats_more_valuable, interval_stats], ignore_index=True)

# Display the results
all_interval_stats_more_valuable

## less valuable team winning

In [ ]:
latest_match_data = load_csv_dataset('create_dataset_1/CSVs we use often/latest_match_data_june_18_SUPERSTAR_INFO_ADDED.csv')
latest_match_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)
latest_match_data['mv_difference_abs'] = latest_match_data['Market.Value.Difference'].abs()
# Assuming 'latest_match_data' is already loaded
latest_match_data['MV_Prediction_Correct'] = np.where(
    (latest_match_data['Market.Value.Difference'] > 0) & (latest_match_data['Score.Difference'] < 0), 1,
    np.where(
        (latest_match_data['Market.Value.Difference'] < 0) & (latest_match_data['Score.Difference'] > 0), 1, 0
    )
)
pd.set_option('display.max_columns', None)

# ... existing code ...

# Create a new column for the year pairs
latest_match_data['year_pair'] = latest_match_data['Season'].apply(lambda x: f"{x//2*2}-{x//2*2+1}")

# Define the intervals
intervals = [0, 500000, 1000000, 5000000, 10000000, 50000000, 100000000, 500000000, 1000000000]
labels = ['0-500K', '500K-1M', '1M-5M', '5M-10M', '10M-50M', '50M-100M', '100M-500M', '500M-1B']

# Create a new column for the intervals
latest_match_data['mv_interval'] = pd.cut(latest_match_data['mv_difference_abs'], bins=intervals, labels=labels)

# Initialize an empty DataFrame to store the results
all_interval_stats_less_valuable = pd.DataFrame()

# Group by the year pairs and intervals and calculate the required statistics
for year_pair, group in latest_match_data.groupby('year_pair'):
    interval_stats = group.groupby('mv_interval').agg(
        total_games=('MV_Prediction_Correct', 'size'),
        more_valuable_wins=('MV_Prediction_Correct', 'sum')
    ).reset_index()
    
    # Calculate the percentage of games won by the more valuable team
    interval_stats['win_percentage'] = (interval_stats['more_valuable_wins'] / interval_stats['total_games']) * 100
    
    # Add the year pair to the results
    interval_stats['year_pair'] = year_pair
    
    # Append to the overall results DataFrame
    all_interval_stats_less_valuable = pd.concat([all_interval_stats_less_valuable, interval_stats], ignore_index=True)

# Display the results
all_interval_stats_less_valuable

## Stacked area plots for each interval over time

In [ ]:
import matplotlib.pyplot as plt
import os

def plot_stacked_area_chart(interval, more_valuable_data, less_valuable_data, save_path):
    # Filter the data for the given interval
    more_valuable_filtered = more_valuable_data[more_valuable_data['mv_interval'] == interval]
    less_valuable_filtered = less_valuable_data[less_valuable_data['mv_interval'] == interval]

    # Merge the data on year_pair
    merged_data = pd.merge(more_valuable_filtered, less_valuable_filtered, on='year_pair', suffixes=('_more', '_less'))

    # Calculate the percentage of draws
    merged_data['draw_percentage'] = 100 - (merged_data['win_percentage_more'] + merged_data['win_percentage_less'])

    # Prepare the data for plotting
    year_pairs = merged_data['year_pair']
    win_more = merged_data['win_percentage_more']
    win_less = merged_data['win_percentage_less']
    draws = merged_data['draw_percentage']
    total_games = merged_data['total_games_more'] + merged_data['total_games_less']

    # Plot the stacked area chart
    plt.figure(figsize=(12, 10))  # Increase the height of the figure
    plt.stackplot(year_pairs, win_more, win_less, draws, labels=['More Valuable Wins', 'Less Valuable Wins', 'Draws'], colors=['#1f77b4', '#ff7f0e', '#2ca02c'])
    plt.legend(loc='lower right')  # Change legend location to bottom right
    plt.xlabel('Year Pair')
    plt.ylabel('Percentage')
    plt.title(f'Win Percentage Over Time for {interval} Interval')
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Add annotations for the number of games
    for i, year_pair in enumerate(year_pairs):
        plt.text(year_pair, 98, f'{total_games.iloc[i]} games', ha='center', va='bottom', fontsize=10, rotation=0)

    # Save the plot as a PNG file
    plt.savefig(os.path.join(save_path, f'win_percentage_{interval}.png'))
    plt.close()

# Define the save path
save_path = '/Users/finneganlaister-smith/Downloads'

# Get the unique intervals from the data
intervals = all_interval_stats_more_valuable['mv_interval'].unique()

# Run the function for each interval
for interval in intervals:
    plot_stacked_area_chart(interval, all_interval_stats_more_valuable, all_interval_stats_less_valuable, save_path)

## Interval charts by year

### more valuable team wins

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import os
import numpy as np

# Custom formatter function to display raw numbers
def custom_formatter(x, pos):
    if x == 1e5:
        return '1e5'
    elif x == 5e5:
        return '5e5'
    elif x == 1e6:
        return '1e6'
    elif x == 5e6:
        return '5e6'
    elif x == 1e7:
        return '1e7'
    elif x == 5e7:
        return '5e7'
    elif x == 1e8:
        return '1e8'
    elif x == 5e8:
        return '5e8'
    elif x == 1e9:
        return '1e9'
    else:
        return f'{x:.0e}'  # fallback for other values

def plot_win_percentage_by_interval(data, save_path):
    # Define the intervals and their corresponding min and max values
    intervals = {
        '0-500K': (1e5, 5e5),
        '500K-1M': (5e5, 1e6),
        '1M-5M': (1e6, 5e6),
        '5M-10M': (5e6, 1e7),
        '10M-50M': (1e7, 5e7),
        '50M-100M': (5e7, 1e8),
        '100M-500M': (1e8, 5e8),
        '500M-1B': (5e8, 1e9)
    }

    # Group by year_pair and plot for each subset
    for year_pair, subset in data.groupby('year_pair'):
        fig, ax = plt.subplots(figsize=(12, 10))

        # Set the x-axis to a logarithmic scale and specify the range from 10^5 to 10^9
        ax.set_xscale('log')
        ax.set_xlim(1e5, 1e9)  # Set the range of x-axis from 10^5 to 10^9

        # Set custom ticks for the x-axis
        ax.set_xticks([1e5, 5e5, 1e6, 5e6, 1e7, 5e7, 1e8, 5e8, 1e9])
        ax.get_xaxis().set_major_formatter(FuncFormatter(custom_formatter))
        ax.tick_params(axis='x', rotation=45)

        # Set the y-axis range from 0 to 100
        ax.set_ylim(0, 100)

        # Initialize a variable to store the previous max interval value and win percentage
        previous_max = None
        previous_win_percentage = None

        # Plot the line for the less valuable team
        for _, row in subset.iterrows():
            interval_min, interval_max = intervals[row['mv_interval']]
            
            # Plot horizontal line segment
            ax.plot([interval_min, interval_max], [row['win_percentage'], row['win_percentage']],
                    label='Less Valuable Team' if previous_max is None else "",  # Label only the first segment to avoid duplicate legend entries
                    color='blue', marker='o', linestyle='-')
            
            # If there's a previous segment, plot a vertical line connecting them
            if previous_max is not None and previous_win_percentage is not None:
                ax.vlines(x=previous_max, ymin=previous_win_percentage, ymax=row['win_percentage'], colors='blue', linestyles='dotted')
            
            # Update previous values
            previous_max = interval_max
            previous_win_percentage = row['win_percentage']

            # Add text label near the right end of the line only if the win percentage changes
            if previous_win_percentage is None or previous_win_percentage != row['win_percentage']:
                ax.text(interval_max, row['win_percentage'], f' Less Valuable Team', verticalalignment='center', color='blue')

            # Check if Total Matches equals 0 and add a pink vertical highlight over the entire range
            if row['total_games'] == 0:
                ax.axvspan(interval_min, interval_max, color='pink', alpha=0.5)  # Highlight the entire interval

            # Add text for non-zero total matches but zero win percentage
            if row['total_games'] > 0 and row['win_percentage'] == 0:
                ax.text(interval_max, row['win_percentage'] - 5, f'<-- {row["total_games"]}G, 0W', verticalalignment='center', color='red')

            # Add text in green for non-zero total matches and non-zero win percentage
            if row['total_games'] > 0 and row['win_percentage'] > 0:
                ax.text(interval_max*10 / np.log(interval_max), row['win_percentage'] + 9,  # Move left by 100000 and up by 5
                        f'<- {row["total_games"]}G\nW%: {row["win_percentage"]:.1f}', 
                        verticalalignment='center', color='green', rotation=82)

        ax.set_xlabel('Interval Range ($)')
        ax.set_ylabel('Win Percentage (%)')
        ax.set_title(f'Win Percentage by Interval Range for More Valuable Team ({year_pair})')

        # Display the legend
        ax.legend()

        # Save the plot as a PNG file
        plt.savefig(os.path.join(save_path, f'win_percentage_{year_pair}_more_valuable.png'))
        plt.close()

# Define the save path
save_path = '/Users/finneganlaister-smith/Downloads'

# Run the function on the data
plot_win_percentage_by_interval(all_interval_stats_more_valuable, save_path)

### less valuable team wins

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import os
import numpy as np

# Custom formatter function to display raw numbers
def custom_formatter(x, pos):
    if x == 1e5:
        return '1e5'
    elif x == 5e5:
        return '5e5'
    elif x == 1e6:
        return '1e6'
    elif x == 5e6:
        return '5e6'
    elif x == 1e7:
        return '1e7'
    elif x == 5e7:
        return '5e7'
    elif x == 1e8:
        return '1e8'
    elif x == 5e8:
        return '5e8'
    elif x == 1e9:
        return '1e9'
    else:
        return f'{x:.0e}'  # fallback for other values

def plot_win_percentage_by_interval(data, save_path):
    # Define the intervals and their corresponding min and max values
    intervals = {
        '0-500K': (1e5, 5e5),
        '500K-1M': (5e5, 1e6),
        '1M-5M': (1e6, 5e6),
        '5M-10M': (5e6, 1e7),
        '10M-50M': (1e7, 5e7),
        '50M-100M': (5e7, 1e8),
        '100M-500M': (1e8, 5e8),
        '500M-1B': (5e8, 1e9)
    }

    # Group by year_pair and plot for each subset
    for year_pair, subset in data.groupby('year_pair'):
        fig, ax = plt.subplots(figsize=(12, 10))

        # Set the x-axis to a logarithmic scale and specify the range from 10^5 to 10^9
        ax.set_xscale('log')
        ax.set_xlim(1e5, 1e9)  # Set the range of x-axis from 10^5 to 10^9

        # Set custom ticks for the x-axis
        ax.set_xticks([1e5, 5e5, 1e6, 5e6, 1e7, 5e7, 1e8, 5e8, 1e9])
        ax.get_xaxis().set_major_formatter(FuncFormatter(custom_formatter))
        ax.tick_params(axis='x', rotation=45)

        # Set the y-axis range from 0 to 100
        ax.set_ylim(0, 100)

        # Initialize a variable to store the previous max interval value and win percentage
        previous_max = None
        previous_win_percentage = None

        # Plot the line for the less valuable team
        for _, row in subset.iterrows():
            interval_min, interval_max = intervals[row['mv_interval']]
            
            # Plot horizontal line segment
            ax.plot([interval_min, interval_max], [row['win_percentage'], row['win_percentage']],
                    label='Less Valuable Team' if previous_max is None else "",  # Label only the first segment to avoid duplicate legend entries
                    color='blue', marker='o', linestyle='-')
            
            # If there's a previous segment, plot a vertical line connecting them
            if previous_max is not None and previous_win_percentage is not None:
                ax.vlines(x=previous_max, ymin=previous_win_percentage, ymax=row['win_percentage'], colors='blue', linestyles='dotted')
            
            # Update previous values
            previous_max = interval_max
            previous_win_percentage = row['win_percentage']

            # Add text label near the right end of the line only if the win percentage changes
            if previous_win_percentage is None or previous_win_percentage != row['win_percentage']:
                ax.text(interval_max, row['win_percentage'], f' Less Valuable Team', verticalalignment='center', color='blue')

            # Check if Total Matches equals 0 and add a pink vertical highlight over the entire range
            if row['total_games'] == 0:
                ax.axvspan(interval_min, interval_max, color='pink', alpha=0.5)  # Highlight the entire interval

            # Add text for non-zero total matches but zero win percentage
            if row['total_games'] > 0 and row['win_percentage'] == 0:
                ax.text(interval_max, row['win_percentage'] - 5, f'<-- {row["total_games"]}G, 0W', verticalalignment='center', color='red')

            # Add text in green for non-zero total matches and non-zero win percentage
            if row['total_games'] > 0 and row['win_percentage'] > 0:
                ax.text(interval_max * 12 / np.log(interval_max), row['win_percentage'] + 15,  # Adjust the value 50000 as needed
                        f'<- {row["total_games"]}G\nwin%: {row["win_percentage"]:.1f}', 
                        verticalalignment='center', color='green', rotation=90)

        ax.set_xlabel('Interval Range ($)')
        ax.set_ylabel('Win Percentage (%)')
        ax.set_title(f'Win Percentage by Interval Range for Less Valuable Team ({year_pair})')

        # Display the legend
        ax.legend()

        # Save the plot as a PNG file
        plt.savefig(os.path.join(save_path, f'win_percentage_{year_pair}_less_valuable.png'))
        plt.close()

# Define the save path
save_path = '/Users/finneganlaister-smith/Downloads'

# Run the function on the data
plot_win_percentage_by_interval(all_interval_stats_less_valuable, save_path)

# RANK

## Lower ranked team winning

In [ ]:
latest_match_data = load_csv_dataset('create_dataset_1/CSVs we use often/latest_match_data_june_18_SUPERSTAR_INFO_ADDED.csv')
latest_match_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)
latest_match_data['rank_difference_abs'] = latest_match_data['Rank.Difference'].abs()
# Assuming 'latest_match_data' is already loaded
latest_match_data['Rank_Prediction_Correct'] = np.where(
    (latest_match_data['Rank.Difference'] > 0) & (latest_match_data['Score.Difference'] > 0), 1,
    np.where(
        (latest_match_data['Rank.Difference'] < 0) & (latest_match_data['Score.Difference'] < 0), 1, 0
    )
)
pd.set_option('display.max_columns', None)

# ... existing code ...

# Create a new column for the year pairs
latest_match_data['year_pair'] = latest_match_data['Season'].apply(lambda x: f"{x//2*2}-{x//2*2+1}")

# Define the intervals
intervals = [0, 5, 10, 15, 30, 60, 90, 120, 150, 180, 210]
labels = ['0-5', '5-10', '10-15', '15-30', '30-60', '60-90', '90-120', '120-150', '150-180', '180-210']

# Create a new column for the intervals
latest_match_data['rank_interval'] = pd.cut(latest_match_data['rank_difference_abs'], bins=intervals, labels=labels)

# Initialize an empty DataFrame to store the results
all_interval_stats_lower_ranked = pd.DataFrame()

# Group by the year pairs and intervals and calculate the required statistics
for year_pair, group in latest_match_data.groupby('year_pair'):
    interval_stats = group.groupby('rank_interval').agg(
        total_games=('Rank_Prediction_Correct', 'size'),
        higher_ranked_wins=('Rank_Prediction_Correct', 'sum')
    ).reset_index()
    
    # Calculate the percentage of games won by the higher ranked team
    interval_stats['win_percentage'] = (interval_stats['higher_ranked_wins'] / interval_stats['total_games']) * 100
    
    # Add the year pair to the results
    interval_stats['year_pair'] = year_pair
    
    # Append to the overall results DataFrame
    all_interval_stats_lower_ranked = pd.concat([all_interval_stats_lower_ranked, interval_stats], ignore_index=True)

# Display the results
all_interval_stats_lower_ranked

## Higher ranked team winning

In [ ]:
latest_match_data = load_csv_dataset('create_dataset_1/CSVs we use often/latest_match_data_june_18_SUPERSTAR_INFO_ADDED.csv')
latest_match_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)
latest_match_data['rank_difference_abs'] = latest_match_data['Rank.Difference'].abs()
# Assuming 'latest_match_data' is already loaded
latest_match_data['Rank_Prediction_Correct'] = np.where(
    (latest_match_data['Rank.Difference'] < 0) & (latest_match_data['Score.Difference'] > 0), 1,
    np.where(
        (latest_match_data['Rank.Difference'] > 0) & (latest_match_data['Score.Difference'] < 0), 1, 0
    )
)
pd.set_option('display.max_columns', None)

# ... existing code ...

# Create a new column for the year pairs
latest_match_data['year_pair'] = latest_match_data['Season'].apply(lambda x: f"{x//2*2}-{x//2*2+1}")

# Define the intervals
intervals = [0, 15, 30, 60, 90, 120, 150, 180, 210]
labels = ['0-15', '15-30', '30-60', '60-90', '90-120', '120-150', '150-180', '180-210']

# Create a new column for the intervals
latest_match_data['rank_interval'] = pd.cut(latest_match_data['rank_difference_abs'], bins=intervals, labels=labels)

# Initialize an empty DataFrame to store the results
all_interval_stats_higher_ranked = pd.DataFrame()

# Group by the year pairs and intervals and calculate the required statistics
for year_pair, group in latest_match_data.groupby('year_pair'):
    interval_stats = group.groupby('rank_interval').agg(
        total_games=('Rank_Prediction_Correct', 'size'),
        higher_ranked_wins=('Rank_Prediction_Correct', 'sum')
    ).reset_index()
    
    # Calculate the percentage of games won by the higher ranked team
    interval_stats['win_percentage'] = (interval_stats['higher_ranked_wins'] / interval_stats['total_games']) * 100
    
    # Add the year pair to the results
    interval_stats['year_pair'] = year_pair
    
    # Append to the overall results DataFrame
    all_interval_stats_higher_ranked = pd.concat([all_interval_stats_higher_ranked, interval_stats], ignore_index=True)

# Display the results
all_interval_stats_higher_ranked

## Stacked area charts by year

In [ ]:
import matplotlib.pyplot as plt
import os

def plot_stacked_area_chart(interval, higher_ranked_data, lower_ranked_data, save_path):
    # Filter the data for the given interval
    higher_ranked_filtered = higher_ranked_data[higher_ranked_data['rank_interval'] == interval]
    lower_ranked_filtered = lower_ranked_data[lower_ranked_data['rank_interval'] == interval]

    # Merge the data on year_pair
    merged_data = pd.merge(higher_ranked_filtered, lower_ranked_filtered, on='year_pair', suffixes=('_higher', '_lower'))

    # Calculate the percentage of draws
    merged_data['draw_percentage'] = 100 - (merged_data['win_percentage_higher'] + merged_data['win_percentage_lower'])

    # Prepare the data for plotting
    year_pairs = merged_data['year_pair']
    win_higher = merged_data['win_percentage_higher']
    win_lower = merged_data['win_percentage_lower']
    draws = merged_data['draw_percentage']
    total_games = merged_data['total_games_higher'] + merged_data['total_games_lower']

    # Plot the stacked area chart
    plt.figure(figsize=(12, 10))  # Increase the height of the figure
    plt.stackplot(year_pairs, win_lower, win_higher, draws, labels=['Lower Ranked Wins', 'Higher Ranked Wins', 'Draws'], colors=['#1f77b4', '#ff7f0e', '#2ca02c'])
    plt.legend(loc='lower right')  # Change legend location to bottom right
    plt.xlabel('Year Pair')
    plt.ylabel('Percentage')
    plt.title(f'Win Percentage Over Time for {interval} Interval')
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Add annotations for the number of games
    for i, year_pair in enumerate(year_pairs):
        plt.text(year_pair, 98, f'{total_games.iloc[i]} games', ha='center', va='bottom', fontsize=10, rotation=0)

    # Save the plot as a PNG file
    plt.savefig(os.path.join(save_path, f'win_percentage_rank_{interval}.png'))
    plt.close()

# Define the save path
save_path = '/Users/finneganlaister-smith/Downloads'

# Get the unique intervals from the data
intervals = all_interval_stats_higher_ranked['rank_interval'].unique()

# Run the function for each interval
for interval in intervals:
    plot_stacked_area_chart(interval, all_interval_stats_higher_ranked, all_interval_stats_lower_ranked, save_path)

## Interval charts by year    

### Lower ranked team wins

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import os

# Custom formatter function to display raw numbers
def custom_formatter(x, pos):
    return f'{int(x)}'  # Display as integer

def plot_win_percentage_by_interval(data, save_path):
    # Define the intervals and their corresponding min and max values
    intervals = {
        '0-15': (0, 15),
        '15-30': (15, 30),
        '30-60': (30, 60),
        '60-90': (60, 90),
        '90-120': (90, 120),
        '120-150': (120, 150),
        '150-180': (150, 180),
        '180-210': (180, 210)
    }

    # Group by year_pair and plot for each subset
    for year_pair, subset in data.groupby('year_pair'):
        fig, ax = plt.subplots(figsize=(12, 10))

        # Set the x-axis to a linear scale and specify the range from 0 to 210
        ax.set_xlim(0, 210)  # Set the range of x-axis from 0 to 210

        # Set custom ticks for the x-axis
        ax.set_xticks([0, 30, 60, 90, 120, 150, 180, 210])
        ax.get_xaxis().set_major_formatter(FuncFormatter(custom_formatter))
        ax.tick_params(axis='x', rotation=45)

        # Set the y-axis range from 0 to 100
        ax.set_ylim(0, 100)

        # Initialize a variable to store the previous max interval value and win percentage
        previous_max = None
        previous_win_percentage = None

        # Plot the line for the higher ranked team
        for _, row in subset.iterrows():
            interval_min, interval_max = intervals[row['rank_interval']]
            
            # Plot horizontal line segment
            ax.plot([interval_min, interval_max], [row['win_percentage'], row['win_percentage']],
                    label='Higher Ranked Team' if previous_max is None else "",  # Label only the first segment to avoid duplicate legend entries
                    color='blue', marker='o', linestyle='-')
            
            # If there's a previous segment, plot a vertical line connecting them
            if previous_max is not None and previous_win_percentage is not None:
                ax.vlines(x=previous_max, ymin=previous_win_percentage, ymax=row['win_percentage'], colors='blue', linestyles='dotted')
            
            # Update previous values
            previous_max = interval_max
            previous_win_percentage = row['win_percentage']

            # Add text label near the right end of the line only if the win percentage changes
            if previous_win_percentage is None or previous_win_percentage != row['win_percentage']:
                ax.text(interval_max, row['win_percentage'], f' Higher Ranked Team', verticalalignment='center', color='blue')

            # Check if Total Matches equals 0 and add a pink vertical highlight over the entire range
            if row['total_games'] == 0:
                ax.axvspan(interval_min, interval_max, color='pink', alpha=0.5)  # Highlight the entire interval

            # Add text for non-zero total matches but zero win percentage
            if row['total_games'] > 0 and row['win_percentage'] == 0:
                ax.text(interval_max, row['win_percentage'] - 5, f'<-- {row["total_games"]}G, 0W', verticalalignment='center', color='red')

            # Add text in green for non-zero total matches and non-zero win percentage
            if row['total_games'] > 0 and row['win_percentage'] > 0:
                ax.text(interval_max - 5, row['win_percentage'] + 3,  # Adjust the position as needed
                        f'<- {row["total_games"]}G\nW%: {row["win_percentage"]:.1f}', 
                        verticalalignment='center', color='green', rotation=0)

        ax.set_xlabel('Interval Range (Rank Difference)')
        ax.set_ylabel('Win Percentage (%)')
        ax.set_title(f'Win Percentage by Interval Range for Higher Ranked Team ({year_pair})')

        # Display the legend
        ax.legend()

        # Save the plot as a PNG file
        plt.savefig(os.path.join(save_path, f'win_percentage_{year_pair}_lower_FIFA_ranked_team.png'))
        plt.close()

# Define the save path
save_path = '/Users/finneganlaister-smith/Downloads'

# Run the function on the data
plot_win_percentage_by_interval(all_interval_stats_lower_ranked, save_path)

### higher ranked team wins

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import os

# Custom formatter function to display raw numbers
def custom_formatter(x, pos):
    return f'{int(x)}'  # Display as integer

def plot_win_percentage_by_interval(data, save_path):
    # Define the intervals and their corresponding min and max values
    intervals = {
        '0-15': (0, 15),
        '15-30': (15, 30),
        '30-60': (30, 60),
        '60-90': (60, 90),
        '90-120': (90, 120),
        '120-150': (120, 150),
        '150-180': (150, 180),
        '180-210': (180, 210)
    }

    # Group by year_pair and plot for each subset
    for year_pair, subset in data.groupby('year_pair'):
        fig, ax = plt.subplots(figsize=(12, 10))

        # Set the x-axis to a linear scale and specify the range from 0 to 210
        ax.set_xlim(0, 210)  # Set the range of x-axis from 0 to 210

        # Set custom ticks for the x-axis
        ax.set_xticks([0, 30, 60, 90, 120, 150, 180, 210])
        ax.get_xaxis().set_major_formatter(FuncFormatter(custom_formatter))
        ax.tick_params(axis='x', rotation=45)

        # Set the y-axis range from 0 to 100
        ax.set_ylim(0, 100)

        # Initialize a variable to store the previous max interval value and win percentage
        previous_max = None
        previous_win_percentage = None

        # Plot the line for the higher ranked team
        for _, row in subset.iterrows():
            interval_min, interval_max = intervals[row['rank_interval']]
            
            # Plot horizontal line segment
            ax.plot([interval_min, interval_max], [row['win_percentage'], row['win_percentage']],
                    label='Higher Ranked Team' if previous_max is None else "",  # Label only the first segment to avoid duplicate legend entries
                    color='blue', marker='o', linestyle='-')
            
            # If there's a previous segment, plot a vertical line connecting them
            if previous_max is not None and previous_win_percentage is not None:
                ax.vlines(x=previous_max, ymin=previous_win_percentage, ymax=row['win_percentage'], colors='blue', linestyles='dotted')
            
            # Update previous values
            previous_max = interval_max
            previous_win_percentage = row['win_percentage']

            # Add text label near the right end of the line only if the win percentage changes
            if previous_win_percentage is None or previous_win_percentage != row['win_percentage']:
                ax.text(interval_max, row['win_percentage'], f' Higher Ranked Team', verticalalignment='center', color='blue')

            # Check if Total Matches equals 0 and add a pink vertical highlight over the entire range
            if row['total_games'] == 0:
                ax.axvspan(interval_min, interval_max, color='pink', alpha=0.5)  # Highlight the entire interval

            # Add text for non-zero total matches but zero win percentage
            if row['total_games'] > 0 and row['win_percentage'] == 0:
                ax.text(interval_max, row['win_percentage'] + 3, f'<-- {row["total_games"]}G, 0W', verticalalignment='center', color='red')

            # Add text in green for non-zero total matches and non-zero win percentage
            if row['total_games'] > 0 and row['win_percentage'] > 0:
                ax.text(interval_max - 5, row['win_percentage'] + 3,  # Adjust the position as needed
                        f'<- {row["total_games"]}G\nW%: {row["win_percentage"]:.1f}', 
                        verticalalignment='center', color='green', rotation=0)

        ax.set_xlabel('Interval Range (Rank Difference)')
        ax.set_ylabel('Win Percentage (%)')
        ax.set_title(f'Win Percentage by Interval Range for Higher Ranked Team ({year_pair})')

        # Display the legend
        ax.legend()

        # Save the plot as a PNG file
        plt.savefig(os.path.join(save_path, f'win_percentage_{year_pair}_higher_FIFA_ranked_team.png'))
        plt.close()

# Define the save path
save_path = '/Users/finneganlaister-smith/Downloads'

# Run the function on the data
plot_win_percentage_by_interval(all_interval_stats_higher_ranked, save_path)